# Лабораторная работа 2

In [27]:
from pymorphy2.shapes import is_roman_number
from pymorphy2 import MorphAnalyzer

from natasha import (
    Segmenter,
    NamesExtractor, 
    MorphVocab,
    DatesExtractor,
    Doc,
    NewsEmbedding,
    NewsNERTagger,
    NewsMorphTagger
)

from yargy import (
    or_,
    not_,
    and_,
    rule,
    Parser
)

from yargy.predicates import (
    in_,
    gte, 
    lte,
    gram,
    is_capitalized,
    custom,
    type
)

from yargy.interpretation import fact
from yargy.relations import gnc_relation
from yargy.pipelines import morph_pipeline


In [28]:
text = """
Шату́ра — город (с 1936 года) областного подчинения (с 2017 года) на востоке Московской области в России, административный центр городского округа Шатура[3] (до 2017 года — Шатурского района). Население — 37 294[2] человек (2023).

Город расположен в Мещёрской низменности, рядом с Шатурской группой озёр. По Егорьевскому шоссе и другим автомобильным дорогам расстояние до МКАД составляет 122 км[4], до центра города Москвы — 138 км[5]. Глава городского поселения Шатура — Алексей Артюхин. Крупнейшие предприятия города — Шатурская ГРЭС и ОАО «Мебельная компания „Шатура“»[6]. Городской праздник — День города Шатуры — традиционно отмечается в первую субботу сентября[7].

Несмотря на то, что Шатура была образована в советское время, город не получил названия, характерного для того периода. Название города имеет исторические корни. В 1417 и 1423 годах в духовных грамотах великого князя Василия Дмитриевича упоминается местность Шатур[8][9][10], позднее в конце XV века в разъезжей грамоте указана Шатурская волость[11]. Впоследствии по местности назван обширный болотный массив, с запасами торфа, — Шатурские болота, которые в свою очередь дали название строящейся электростанции и целому ряду посёлков: Шатурторф, Шатурстрой и Шатура[12]. Первоначальная форма названия с ударением на первом слоге сохранилась в названии села Ша́тур, расположенного в 15 км к юго-западу от города. Посёлок строителей электростанции ещё имел в своём названии исходное имя — Шатурстрой, но сама электростанция уже называлась Малая Шатура, а образованный в 1928 году рабочий посёлок — Шатура[13].

Название происходит от местного гидронима Шатур[12]. В пользу этой теории можно привести ещё два гидронима с аналогичным названием: озеро Шатурское (Шатурково), расположенное на юго-востоке района, и речка Шатурка (Шатур), обозначенная на карте Меленковского уезда Владимирской губернии XVIII века и на межевой карте А. И. Менде 1850 года[14][15]. Кроме того, в Шатурском и Егорьевском районах и в регионе Среднего Поочья есть и другие гидронимы на -ур — Кендур, Синур, Нинур и т. д., что позволяет предположить их общее происхождение. Эти гидронимы, предположительно, принадлежат к пласту дорусской субстратной топонимии. Однако определить их принадлежность к финно-угорским или дофинно-угорским языкам пока невозможно[16].

Шатура расположена в пределах Мещёрской низменности, относящейся к Восточно-Европейской равнине. К северу от города находится Шатурская озёрная группа, состоящая из 5 озёр: Святое, Муромское, Белое, Чёрное и Чёрное-Спасское[17]. Город расположен в 138 км к востоку от Москвы (122 км от МКАД), связан со столицей автомагистралью Р106 (Куровское — Шатура — Кривандино — Дмитровский Погост) и Р105 (Егорьевским шоссе). Через город проходит железнодорожная линия Казанского направления.

Территория города Шатуры находится в центральной части Восточно-Европейской платформы. На глубине более 1000 м лежит Русская плита, образовавшаяся в древнее докембрийское время. Древние отложения (мезозойские, юрские, меловые), перекрыты четвертичными, которые связаны в основном с деятельностью ледника, наносами (моренными и флювиогляциальными). Близкое залегание водоупорных юрских глин, плоский рельеф и слабый дренаж обусловили наличие многочисленных болот и торфяников[19].

Город находится на территории Мещёрской физико-географической провинции подзоны смешанных лесов лесной зоны с преобладанием ландшафтов водно-ледниковых равнин. Рельеф плоский и волнистый, среди форм рельефа преобладают песчаные бугры (гривы) и сырые, заболоченные котловины. Почвы торфяные верховые и низинные, а также подзолистые и подзолисто-глееватые[20]. Средняя высота центра города — 125 м над уровнем моря[1].

Климат умеренно континентальный. Континентальность климата выражена сильнее, чем в западных частях Московской области; для Шатуры, как и для района, характерна более низкая температура зимой и более высокая температура летом. Среднегодовая температура воздуха +4,5°C; средняя температура января -9,6°C; средняя температура июля +18,4°C. Абсолютный максимум в +39°C наблюдался в августе 2010 года, а абсолютный минимум (за период с 1960 года) в -43°C — в январе 1987-го.[21].

Средняя годовая норма осадков составляет 570 мм. Наибольшее их количество обычно приходится на июнь — август, а наименьшее — на март[22].

Весенние заморозки обычно заканчиваются к 10-15 мая. Первые заморозки начинаются со второй половины сентября, в отдельные годы — в конце августа. Появление снега наблюдается в период 27 октября — 4 ноября, а устойчивый снежный покров образуется в третью декаду ноября (23-25). Начало разрушения снежного покрова приходится на вторую декаду апреля. Число дней со снежным покровом составляет 150—155 дней[22].

В течение года преобладающими ветрами являются юго-восточные и западные. В зимние месяцы преобладают юго-западные и южные ветры. Средняя скорость ветра за год — 3 м/сек[22].

В Шатуре, как и во всём районе, преобладает пасмурная погода. Ясные дни наблюдаются в течение года в среднем лишь 1-2 раза в месяц. Зимой пасмурное небо наблюдается в среднем более 20 дней в месяц, летом — 10-12 в месяц[23].

Экологическая обстановка в Шатуре и её пригородах оценивается как напряжённая[24]. Наибольшее локальное воздействие на окружающую среду оказывает Шатурская ГРЭС[25]. В связи с наличием в городе крупной электростанции Шатура занимает одно из первых мест в Московской области по доле выбросов от стационарных источников, загрязняющих приземный слой атмосферы[26]. В почвах на территории города выявлены техногенные аномалии цинка, меди и свинца. Кроме того, в окрестностях Шатуры имеются зоны с повышенной концентрацией кадмия, не превышающей предельно допустимой концентрации[27]. К югу от Шатуры находится закрытый летом 2020 года полигон ТБО[28].

Шатура окружена большим количеством торфяников, наличие которых приводит к торфяным пожарам в летний период. Особенно сильные торфяные пожары отмечались в 1972, 1981, 1992, 2002 и 2010 годах. Пожары приводят к сильному задымлению воздуха. В состав смога входит угарный газ, мелкие взвешенные частицы, бензол и другие продукты горения[29]. Наиболее близкий к Шатуре очаг возгорания находится в Малькиной Гриве[30]. С 2010 года в Московской области реализовывалась долгосрочная целевая программа «Экология Подмосковья на 2011—2013 гг.», согласно которой в Шатурском районе проводилось обводнение торфяников[31]. К концу 2013 года программа обводнения была выполнена в полном объёме[32].

В Шатуре имеется городской парк культуры и отдыха им. Ю. А. Гагарина, на территории которого расположены аттракционы (в том числе, колесо обозрения), детские площадки и летнее кафе. В парке регулярно проводятся культурно-массовые мероприятия, такие как День Победы, День памяти и скорби, День города. Парк занимает территорию площадью 8 га[33].

Шатура, как и вся Московская область, находится в часовой зоне МСК (московское время). Смещение применяемого времени относительно UTC составляет +3:00[34].

По берегам озера Святого обнаружено несколько неолитических стоянок и поселений, относящихся к верхневолжской, льяловской и волосовской археологическим культурам[35]. Археологические находки свидетельствуют о том, что окрестности города были заселены ещё много тысячелетий назад. До IX—X веков данную территорию населяли в основном финно-угорские племена, предположительно мещёра[36]. Заселение славянами началось с IX—XI вв.[37]

Во времена Киевской Руси территория современной Шатуры находилась в Ростово-Суздальском княжестве (с середины XII века во Владимиро-Суздальском)[38][39], а к концу XIV века вошла в состав Московского княжества[40]. В писцовой книге Владимирского уезда 1637—1648 гг. местность южнее озера Святого относится к Шатурской волости Владимирского уезда, там же упоминается деревня Перинская (ныне улица Ботино)[41].

В 1708 году образована Московская губерния, куда вошла территория города[42]. После образования в 1719 году провинций данная местность вошла во Владимирскую провинцию, а с 1727 года во вновь восстановленный Владимирский уезд. В 1778 году образовано Рязанское наместничество (с 1796 года губерния). Впоследствии вплоть до начала XX века территория города входила в Егорьевский уезд Рязанской губернии.

В 1793 году впервые упоминается деревня Тарбеиха[43]. Несмотря на то, что деревня располагалась на Владимирском тракте[44], она была сравнительно небольшой и в ней отсутствовали какие-либо общественные заведения. Мужчины занимались в основном плотничеством, а женщины ткали нанку[45][46]. Однако ситуация меняется в начале XX века. Так в 1912 году открыто движение по железнодорожной ветке Люберцы — Арзамас. Железная дорога прошла к югу от деревни Тарбеихи[47].

Изыскательские работы на торфяных болотах Мещёрского края велись ещё до первой мировой войны. В добыче торфа участвовали местные фабриканты Хлудовы, Клоповы, Морозовы, Костеревы и др. Торф использовался на фабриках Егорьевска и Орехово-Зуева[48]. В 1914 году Р. Э. Классон, сотрудник «Общества электрического освещения 1886 года», исследует шатурские торфяные болота[49].

После февральской революции Московская городская управа выделила небольшие средства на разработку шатурских торфяных болот. Работами по подготовке строительства электростанции руководили А. В. Винтер и И. И. Радченко[49]. Однако вскоре кредитование было прекращено и стройка остановлена. За лето 1917 года удалось только произвести некоторые изыскательские работы и заготовить несколько десятков столбов для электролиний[50][51].

В декабре 1917 года А. В. Винтер сделал краткий доклад главе государства В. И. Ленину о проекте сооружения на шатурских болотах крупной электрической станции на торфяном топливе с целью обеспечения Москвы электроэнергией. Ленин одобрил данный проект и отдал распоряжение о начале строительства станции. Впоследствии А. В. Винтер вспоминал: «Декабрьский день 1917 г. в Смольном следует считать первым днём строительства Шатурской электростанции»[52].

20 апреля 1918 года Совнарком принял декрет «Декрет о разработках торфяного топлива»[53] и положение «Об организации Главного торфяного комитета»[54]. Так была заложена нормативная база для организации торфоразработок. Спустя месяц был создан штаб по строительству Шатурской электростанции. И. И. Радченко был назначен начальником Главторфа, а А. В. Винтер — начальником строительства[50]. Весной этого же года образовано предприятие «Шатурские государственные торфяные разработки Главного торфяного комитета ВСНХ РСФСР» (впоследствии Шатурское торфопредприятие)[55].

2 мая 1918 года Винтер выбрал на восточном берегу Чёрного озера место под строительство электростанции[57]. Здесь была построена контора и несколько бараков, а в 1919 году возникает посёлок Шатурстрой («Учебный Городок»)[58]. В августе 1919 года началось строительство временной электростанции (Малая Шатура), и уже 25 июня 1920 года состоялось её торжественное открытие, на котором присутствовал Председатель ВЦИК М. И. Калинин[59][60].

К 1920 году к западу от Чёрного озера был построен жилой посёлок, где располагались бараки, жилые дома, школа, баня и столовые[61]. В 1922 году посёлок получил название Чёрное Озеро[60].

В конце июня 1923 года началось строительство Большой Шатурской ГРЭС. 23-09-1925 станция была включена в сеть МОГЭС, а 6-12-1925 года электростанция вступила в строй. На митинге, посвящённом открытию ГРЭС, ей было присвоено имя В. И. Ленина[62].

За время строительства Шатурской электростанции была создана огромная инфраструктура, образованы десятки посёлков. Однако вплоть до 1928 года на всей этой территории отсутствовала единая официальная администрация, фактически вся административная власть была сосредоточена в руках Винтера[63].

21 мая 1928 года рабочий посёлок Чёрное Озеро, посёлок Шатурстрой, Временный посёлок и деревня Тарбеиха были объединены в рабочий посёлок Шатура[65][58]. Первым председателем поселкового совета был Г. И. Безруков, последним председателем — И. С. Ершов[66]. С 1928 года центр общественной и хозяйственной жизни перемещается из Центрального посёлка (ныне Шатурторф) в Шатуру. Здесь были построены жилые дома, клуб и административные здания. Типография газеты также переезжает в Шатуру, и в апреле 1930 года вышел первый номер газеты «Ленинская Шатура»[67].

В 1929 году рабочий посёлок стал центром вновь образованного Шатурского района. Однако уже в 1933 году Шатурский район был ликвидирован, но Шатура с прилегающей сельской территорией была выделена в особую административно-территориальную единицу[68].

В 1932 году создан Шатурский государственный торфяной трест (впоследствии ПО «Шатурторф»)[69].

17 апреля 1936 года рабочий посёлок Шатура получил статус города[66]. Первым председателем стал Шилов В. С. (25.05.1936 — 03.06.1937), затем непродолжительное время исполняющим обязанности председателя был Кузьмин Ф. В. С 1 сентября 1937 года на данный пост утверждён Ратников Михаил Антонович, который руководил горисполкомом вплоть до начала Великой Отечественной войны[70].

В годы Великой Отечественной войны районные электростанции Московской энергосистемы были эвакуированы. Основная нагрузка по обеспечению столицы электроэнергией легла на ГРЭС им. Классона, Шатурскую ГРЭС и ТЭЦ в Орехово-Зуеве[71]. Электростанцию и железную дорогу защищали зенитные орудия, которые стояли в Шатуре и её окрестностях, а также вдоль железной дороги[72]. Налёты немецкой авиации происходили с октября 1941 по июнь 1943 года, однако за всё время ни одна бомба не упала на Шатурскую ГРЭС. За выполнение боевых задач по защите электростанции 232-му зенитно-артиллерийскому дивизиону ПВО было вручено Боевое Знамя. С 5 по 19 декабря 1941 года в Шатуре размещался военно-полевой госпиталь 2307[73].

За годы войны в ряды Красной Армии из Шатуры и района было призвано 33 860 человек, 10 108 из них не вернулись с полей сражений. Одиннадцать шатурян удостоены звания Героя Советского Союза[74]. На электростанции и торфяных предприятиях мужчин, ушедших на фронт, заменили женщины, старики и подростки. Тогда в Шатуре зародилось движение двухсотниц, а затем пятисотниц и шестисотниц. Женщины выполняли дневную норму по заготовке торфа на 500% и 600%[75].

В 1956 году вновь образован Шатурский район, в состав которого вошли населённые пункты, подчинённые Шатурскому горсовету, и упразднённый Кривандинский район.

В 1966—1972 годах на электростанции были введены три новых блока, в результате мощность ГРЭС возросла в 5 раз[76]. Во второй половине XX века развиваются также отрасли народного хозяйства не связанные с энергетикой и добычей торфа. В 1961 году открыта Шатурская мебельная фабрика, преобразованная в 1963 году в мебельный комбинат. В 1979 году в Шатуре основан Научно-исследовательский центр по технологическим лазерам (НИЦ ТЛ) АН СССР[77]. Кроме того, в 1960-70-х годах построены новая швейная фабрика, хлебокомбинат, молокозавод, горпищекомбинат, завод по ремонту телеаппаратуры[78].

В 80-х годах XX века объём торфоразработок постепенно снижается, и в 1986—1989 годы был реализован проект «Реконструкция ГРЭС № 5 на сжигание газа»[79].

В 2004 году в состав города Шатура были включены посёлок городского типа Керва и деревня Ботино[80].

В 2005 году было создано муниципальное образование «Городское поселение Шатура», включившее город Шатуру и 23 сельских населённых пункта[81].

19 июня 2017 года город будет преобразован из города районного подчинения в город областного подчинения и вместе с административной территорией станет административно-территориальной единицей вместо Шатурского района[82].

По данным Всероссийской переписи населения 2020 года, на 1 октября 2021 года по численности населения город находился на 404-м месте из 1120[103] городов Российской Федерации[104]..

С момента образования Шатуры и на протяжении всего XX века происходил постоянный рост численности населения города. В период 1989—2008 гг. численность постоянного населения практически не менялась[28]. С 2010 года снова наблюдается незначительный рост численности жителей. Однако основной прирост населения происходит за счёт миграционного притока, в городе же смертность превышает рождаемость более чем в два раза[28]. Следует отметить достаточно высокую концентрацию населения: доля Шатуры в численности населения района в 2014 году составляла 46%.

По данным переписи 2010 года численность населения Шатуры составила 32 885 человека, в том числе 14 917 мужчин и 17 968 женщины[105]. На 2008 год доля трудоспособного населения составляла около 58%, старше трудоспособного — 24,2%, моложе трудоспособного — 18,2%. В сравнении с 1989 годом отмечается увеличение удельного веса населения старше трудоспособного возраста и уменьшение населения младше трудоспособного возраста. Однако не всё трудоспособное население работает в городе, для Шатуры характерна маятниковая трудовая миграция — 8 % от населения города каждый день ездят в Москву и другие соседние города на работу[28].

Первое изображение эмблемы Шатуры появилось в 1925 году на чертежах шатурского строительства. На двух ржаных снопах была изображена электростанция, от которой отходили лучи. По краям электростанции стояли рабочий-энергетик и рабочий-торфяник. Однако впоследствии эмблема в качестве официального символа не использовалась. В 1957 году художник А. И. Семёнов предложил в качестве символа изображение опоры «шатурского типа». Данное изображение стало неофициальной эмблемой газеты «Ленинская Шатура», а потом в натуральную величину была поставлена дорожным указателем на въезде в город[106].

Первый официальный герб Шатуры был утверждён 16 ноября 1995 года решением главы администрации Шатурского района. Автором герба была Мария Анатольевна Ерёмина. Герб представлял собой щит, разделённый горизонтально на две части. В верхнем червлёном поле герба был изображён Святой Георгий на коне серебряного цвета, колющий копьём золотого змея. Нижнее поле зелёного цвета пересекалось пятью волнистыми лазоревыми поясами. Поверх поля изображался червлёный круг, в центре которого размещалась электроопора со стрелой. По сторонам червлёного круга были изображены золотой колос и часть зубчатого колеса, внизу имелась надпись лазоревыми цифрами 1936, год образования города. Верхняя часть щита обозначала административно-территориальную принадлежность Шатуры к Московской области, нижняя — символизировала Шатурскую ГРЭС[107].

Ныне действующий герб утверждён решением Совета депутатов городского поселения Шатура от 26 мая 2011 года № 6/19 «О гербе городского поселения Шатура Шатурского муниципального района Московской области», и впоследствии внесён в Государственный геральдический регистр Российской Федерации за № 7169[108]. Герб городского поселения Шатура разработан на основе герба Шатурского района. Описание герба: «В зелёном поле над оконечностью, пересечённой мурованным золотом и лазурью, летящий вправо и вверх с воздетыми крыльями золотой журавль, сопровождаемый выходящим в правом нижнем углу пламенеющим солнцем (без изображения лица) того же металла»[109].

Ныне действующий флаг утверждён решением Совета депутатов городского поселения Шатура от 26 мая 2011 года № 7/19 «О флаге городского поселения Шатура Шатурского муниципального района Московской области». Флаг в целом повторяет оформление герба. Описание флага: «Прямоугольное двухстороннее зелёное полотнище с отношением ширины к длине 2:3, воспроизводящее композицию герба городского поселения Шатура голубым и жёлтым цветами»[110].

В городе крупный спортивный комплекс, включающий бассейн, стадион, несколько открытых площадок, построенный в 2007 физкультурно—оздоровительный комплекс (ФОК «Шатура», c бассейном).

На данный момент Городской округ Шатура на первенство Московской области по футболу среди мужских команд представляет местный амбициозный футбольный клуб «Сварог».

В городе расположен футбольный клуб «Селтик»[112], выступающий в Первенстве России среди команд III дивизиона (ЛФЛ) (зона «Московская область», группа Б), Чемпионате, Первенстве и Кубке городского округа Шатуры, а также в Лиге чемпионов Подмосковья[113]. Ранее в любительском первенстве России (ЛФЛ/КФК) город представляли команды «Динамо»[114] и «Энергия»/ФК «Шатура»[115].

Крупнейшие предприятия города:
1. Шатурская ГРЭС — крупная электростанция регионального значения по выработке электрической и тепловой энергии
2. МК «Шатура» — крупнейший в России комбинат по производству мебели
3. ООО «Шаттдекор» — производство мебельной облицовочной плёнки.

Город расположен на южном, через Казань, направлении транссибирской железнодорожной магистрали. Со станции Шатура отправляются пассажирские электропоезда на Москву, время в пути до Казанского вокзала около 2 час. 20 мин, до станции метро Выхино около 2 час. 00 мин, до станции Люберцы-1 примерно 1 час. 50 мин. Прямое сообщение с городами Куровское и Люберцы. В направлении от Москвы со станции Шатура электропоезда идут до станции Черусти (около 30 мин).

Через город проходит автодорога регионального значения Р106 Куровское — Шатура — Кривандино — Дмитровский Погост — пересечение с автодорогой Р105 (Москва — Егорьевск — Касимов). Регулярное прямое автобусное сообщение с Москвой (м. Котельники), с Рязанью, Спас-Клепиками, Егорьевском, Рошалем, Орехово-Зуевым, городами и посёлками района. Городской транспорт представлен шестью автобусными маршрутами. В городе находится станция Чёрное озеро узкоколейной железной дороги ОАО «Шатурторф» (ныне демонтированной).

Самыми крупными млекопитающими округа являются лось и кабан, изредка встречаются рысь, барсук и бурый медведь. Из хищников распространены волки, лисы, речные выдры, лесные куницы, хорьки, горностаи и ласки. Много мышевидных грызунов, таких как серая крыса, мышь домовая, мышь лесная, мышь полевая, мышь-малютка. Из насекомоядных видов распространены ёж, крот, землеройка, в очень малочисленном виде сохранилась выхухоль. Встречаются несколько видов летучих мышей, среди которых наиболее распространены рыжая летучая мышь и ушан. Повсеместно водятся белки и зайцы (беляк и русак)[35].

Орнитофауна представлена более широко, чем млекопитающие. В хвойных лесах сохранился глухарь, чёрный дятел, некоторые виды крупных неясытей. В смешанных и лиственных лесах встречаются рябчики, вальдшнепы, сойки, большой пёстрый дятел, дикие голуби, некоторые виды дроздов. На водоёмах гнездятся утки и кулики. В поймах рек и заболоченных местах встречаются чирки, крачки, чибисы и болотные совы. На больших озёрах водятся чайки[35]. Из хищных птиц встречаются сова, филин и ястреб. На зимовку прилетают снегири и клесты[34].

Из пресмыкающихся встречается ядовитая гадюка, уж, медяница, живородящая и прыткая ящерицы. Земноводные представлены лягушкой, обыкновенной жабой, гребенчатым и обыкновенным тритоном[35].

В многочисленных озёрах и реках округа обитают карась, линь, бычок-ротан, плотва, лещ, судак, щука, окунь. В некоторых озёрах встречаются раки[35].

3 ноября 1943 года в подмосковной Шатуре родился советский актер Валерий Цветков. Он известен как исполнитель ролей генералов в фильмах, посвященных Отечественной войне: "День командира дивизии" (1983), "Атака" (1986)"Сталинград" (1989).

Памятники монументального искусства
Памятник «Воинам-шатурянам 1941—1945 гг.».
Бюст А. В. Винтера.
Монумент «Космонавты» в сквере XXX лет Октября.
Мемориал «Аллея героев». Установлен в честь шатурян, Героев Советского Союза.
Мемориал «Воинам Великой Отечественной войны — работникам ГРЭС № 5».
Мемориал «Воинам-торфяникам».
Памятник «Торфяница».
Памятник «Зенитка». В честь зенитчиков 232 ОЗАД.
Мемориал «Воинам-интернационалистам».
Мемориал «Жертвам политических репрессий».
Мемориал «Ликвидаторам Чернобыля»[118].
"""

In [29]:
morph_vocab = MorphVocab()
segmenter = Segmenter()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
ner_tagger = NewsNERTagger(emb)

doc = Doc(text)

#### Сегментация

In [30]:
doc.segment(segmenter)
doc.tokens[:3]

[DocToken(start=1, stop=8, text='Шату́ра'),
 DocToken(start=9, stop=10, text='—'),
 DocToken(start=11, stop=16, text='город')]

#### Определение части речи

In [31]:
doc.tag_morph(morph_tagger)
doc.tokens[:5]

[DocToken(start=1, stop=8, text='Шату́ра', pos='PROPN', feats=<Inan,Nom,Masc,Sing>),
 DocToken(start=9, stop=10, text='—', pos='PUNCT'),
 DocToken(start=11, stop=16, text='город', pos='NOUN', feats=<Inan,Nom,Masc,Sing>),
 DocToken(start=17, stop=18, text='(', pos='PUNCT'),
 DocToken(start=18, stop=19, text='с', pos='ADP')]

#### Лемматизация

In [32]:
for token in doc.tokens:
    token.lemmatize(morph_vocab)
doc.tokens[:5]

[DocToken(start=1, stop=8, text='Шату́ра', pos='PROPN', feats=<Inan,Nom,Masc,Sing>, lemma='шату́р'),
 DocToken(start=9, stop=10, text='—', pos='PUNCT', lemma='—'),
 DocToken(start=11, stop=16, text='город', pos='NOUN', feats=<Inan,Nom,Masc,Sing>, lemma='город'),
 DocToken(start=17, stop=18, text='(', pos='PUNCT', lemma='('),
 DocToken(start=18, stop=19, text='с', pos='ADP', lemma='с')]

#### Разметка именованных сущности

In [33]:
doc.tag_ner(ner_tagger)
doc.ner.print()

Шату́ра — город (с 1936 года) областного подчинения (с 2017 года) на 
LOC────                                                              
востоке Московской области в России, административный центр городского
        LOC───────────────   LOC───                                   
 округа Шатура[3] (до 2017 года — Шатурского района). Население — 37 
        LOC───                    LOC──────────────                  
294[2] человек (2023).
Город расположен в Мещёрской низменности, рядом с Шатурской группой 
                   LOC──────────────────          ORG───────────────
озёр. По Егорьевскому шоссе и другим автомобильным дорогам расстояние 
────     LOC───────────────                                           
до МКАД составляет 122 км[4], до центра города Москвы — 138 км[5]. 
   LOC─                                        LOC───              
Глава городского поселения Шатура — Алексей Артюхин. Крупнейшие 
                           LOC───   PER────────────             
предприят

#### Выделение дат

In [34]:
extractor = DatesExtractor(morph_vocab)
matches = [match.fact for match in extractor(text)]
matches

[Date(
     year=1936,
     month=None,
     day=None
 ),
 Date(
     year=2017,
     month=None,
     day=None
 ),
 Date(
     year=2017,
     month=None,
     day=None
 ),
 Date(
     year=1423,
     month=None,
     day=None
 ),
 Date(
     year=1928,
     month=None,
     day=None
 ),
 Date(
     year=1850,
     month=None,
     day=None
 ),
 Date(
     year=2010,
     month=8,
     day=None
 ),
 Date(
     year=1960,
     month=None,
     day=None
 ),
 Date(
     year=1987,
     month=1,
     day=None
 ),
 Date(
     year=None,
     month=5,
     day=15
 ),
 Date(
     year=None,
     month=10,
     day=27
 ),
 Date(
     year=None,
     month=11,
     day=4
 ),
 Date(
     year=2020,
     month=None,
     day=None
 ),
 Date(
     year=2010,
     month=None,
     day=None
 ),
 Date(
     year=2010,
     month=None,
     day=None
 ),
 Date(
     year=2013,
     month=None,
     day=None
 ),
 Date(
     year=2013,
     month=None,
     day=None
 ),
 Date(
     year=1648,
     month=

In [35]:
# не поддерживается обработка дат, которые задаются в формате yyyy-MM-dd, dd-MM-yyyy

Date = fact(
    'Date',
    ['year', 'month', 'day']
)

YEAR = and_(
    gte(1),
    lte(2100)
).interpretation(
    Date.year
)

MONTH = and_(
    gte(1),
    lte(12)
).interpretation(
    Date.month
)

DAY = and_(
    gte(1),
    lte(31)
).interpretation(
    Date.day
)


DT = or_(
    rule(
        YEAR,
        '-',
        MONTH,
        '-',
        DAY
    ),
    rule(
        DAY,
        '-',
        MONTH,
        '-',
        YEAR
    )
).interpretation(Date)

parser = Parser(DT)
matches = [match.fact for match in parser.findall(text)]
matches

[Date(
     year='1925',
     month='09',
     day='23'
 ),
 Date(
     year='1925',
     month='12',
     day='6'
 )]

#### Выделение временных отрезков в веках

In [36]:
Century = fact(
    'Century',
    ['value']
)

CENTURY_PIPELINE = morph_pipeline([
    'век',
    'в',
    'столетие'
])

CENTURY = rule(
    custom(lambda token: set(token).issubset('IVXLCDM')).interpretation(Century.value),
    CENTURY_PIPELINE
).interpretation(Century)

parser = Parser(CENTURY)
matches = [match.fact for match in parser.findall(text)]
matches


[Century(
     value='XV'
 ),
 Century(
     value='XVIII'
 ),
 Century(
     value='X'
 ),
 Century(
     value='XI'
 ),
 Century(
     value='XII'
 ),
 Century(
     value='XIV'
 ),
 Century(
     value='XX'
 ),
 Century(
     value='XX'
 ),
 Century(
     value='XX'
 ),
 Century(
     value='XX'
 ),
 Century(
     value='XX'
 )]

#### Выделение имён

In [37]:
extractor = NamesExtractor(morph_vocab)
matches = [match.fact for match in extractor(text)]
matches

[Name(
     first='Шату',
     last=None,
     middle=None
 ),
 Name(
     first=None,
     last='город',
     middle=None
 ),
 Name(
     first=None,
     last='с',
     middle=None
 ),
 Name(
     first=None,
     last='с',
     middle=None
 ),
 Name(
     first=None,
     last='Московской',
     middle=None
 ),
 Name(
     first=None,
     last='в',
     middle=None
 ),
 Name(
     first=None,
     last='центр',
     middle=None
 ),
 Name(
     first=None,
     last='городского',
     middle=None
 ),
 Name(
     first=None,
     last='Шатура',
     middle=None
 ),
 Name(
     first=None,
     last='до',
     middle=None
 ),
 Name(
     first=None,
     last='Шатурского',
     middle=None
 ),
 Name(
     first=None,
     last='Город',
     middle=None
 ),
 Name(
     first=None,
     last='в',
     middle=None
 ),
 Name(
     first=None,
     last='Мещёрской',
     middle=None
 ),
 Name(
     first=None,
     last='с',
     middle=None
 ),
 Name(
     first=None,
     last='Шатурской

In [38]:
# требуется вычленять ФИО, приведённый выше подход ошибочно добавляет много "мусора"
# это нужно исправить
# необходимо идентифицировать следующие форматы: Иванов Иван Иванович, Иван Иванович Иванов, Иван Иванов, Иванов Иван, И. И. Иванов, Иванов И. И., И. Иванов, Иванов И.

Name = fact(
    'Name',
    ['first', 'last', 'middle']
)

gnc = gnc_relation()

NAME = and_(
    gram('Name'),
    is_capitalized()
    ).interpretation(Name.first.inflected()).match(gnc)
PATR = and_(
    gram('Patr'),
    is_capitalized()
    ).interpretation(Name.middle.inflected()).match(gnc)
SURN = and_(
    gram('Surn'),
    is_capitalized()
    ).interpretation(Name.last.inflected()).match(gnc)

NAME_ABBR = and_(
    gram('Abbr'),
    is_capitalized()
    ).interpretation(Name.first.inflected())
PART_ABBR = and_(
    gram('Abbr'),
    is_capitalized()
    ).interpretation(Name.middle.inflected())

NAME_FULL = or_(
    rule(
        NAME,
        PATR,
        SURN
    ),
    rule(
        SURN,
        NAME,
        PATR
    ),
    rule(
        NAME,
        SURN
    ),
    rule(
        SURN,
        NAME
    ),
    rule(
        NAME_ABBR,
        '.',
        PART_ABBR,
        '.',
        SURN
    ),
    rule(
        SURN,
        NAME_ABBR,
        '.',
        PART_ABBR,
        '.'
    ),
    rule(
        NAME_ABBR,
        '.',
        SURN
    ),
    rule(
        SURN,
        NAME_ABBR,
        '.'
    )
).interpretation(Name)

parser = Parser(NAME_FULL)
matches = [match.fact for match in parser.findall(text)]
matches

[Name(
     first='алексей',
     last='артюхин',
     middle=None
 ),
 Name(
     first='ю',
     last='гагарин',
     middle='а'
 ),
 Name(
     first='и',
     last='радченко',
     middle='и'
 ),
 Name(
     first='в',
     last='ленин',
     middle='и'
 ),
 Name(
     first='и',
     last='радченко',
     middle='и'
 ),
 Name(
     first='м',
     last='калинин',
     middle='и'
 ),
 Name(
     first='в',
     last='ленин',
     middle='и'
 ),
 Name(
     first='г',
     last='безруков',
     middle='и'
 ),
 Name(
     first='и',
     last='ершовый',
     middle='с'
 ),
 Name(
     first='в',
     last='шилов',
     middle='с'
 ),
 Name(
     first='ф',
     last='кузьмин',
     middle='в'
 ),
 Name(
     first='михаил',
     last='ратников',
     middle='антонович'
 ),
 Name(
     first='а',
     last='семёнов',
     middle='и'
 ),
 Name(
     first='мария',
     last='ерёмина',
     middle='анатольевна'
 ),
 Name(
     first='валерий',
     last='цветков',
     middle=None
 )]

#### Выделение имён с ассоциированными профессиями

In [39]:
Person = fact(
    'Person',
    ['profession', 'name']
)

PROFESSION_PIPELINE = morph_pipeline([
    'глава государства', 
    'начальник',
    'председатель', 
    'художник', 
    'актер'
])

PERSON = or_(
    rule(
        PROFESSION_PIPELINE.interpretation(Person.profession.inflected()),
        gram('ADJF').optional().repeatable(),
        gram('NOUN').optional(),
        gram('VERB').optional(),
        NAME_FULL.interpretation(Person.name)
    ),
    rule(
        NAME_FULL.interpretation(Person.name),
        gram('ADJF').optional().repeatable(),
        gram('NOUN').optional(),
        gram('VERB').optional(),
        PROFESSION_PIPELINE.interpretation(Person.profession.inflected())
    )
).interpretation(Person)

parser = Parser(PERSON)
matches = [match.fact for match in parser.findall(text)]
matches

[Person(
     profession='глава государство',
     name=Name(
         first='в',
         last='ленин',
         middle='и'
     )
 ),
 Person(
     profession='председатель',
     name=Name(
         first='м',
         last='калинин',
         middle='и'
     )
 ),
 Person(
     profession='председатель',
     name=Name(
         first='г',
         last='безруков',
         middle='и'
     )
 ),
 Person(
     profession='председатель',
     name=Name(
         first='в',
         last='шилов',
         middle='с'
     )
 ),
 Person(
     profession='председатель',
     name=Name(
         first='ф',
         last='кузьмин',
         middle='в'
     )
 ),
 Person(
     profession='художник',
     name=Name(
         first='а',
         last='семёнов',
         middle='и'
     )
 ),
 Person(
     profession='актёр',
     name=Name(
         first='валерий',
         last='цветков',
         middle=None
     )
 )]

#### Выделение названий памятников и мемориалов

In [40]:
Museum = fact(
    'Museum',
    ['type', 'name']
)

MUSEUM_PIPELINE = morph_pipeline([
    'памятник',
    'бюст',
    'мемориал',
    'монумент'
])

MEM_PLACE = rule(
    MUSEUM_PIPELINE.interpretation(Museum.type),
    rule(
        in_('«"'),
        not_(in_('»')).repeatable(),
        in_('»"')
    ).interpretation(Museum.name)
).interpretation(Museum)

parser = Parser(MEM_PLACE)
matches = [match.fact for match in parser.findall(text)]
matches

[Museum(
     type='Памятник',
     name='«Воинам-шатурянам 1941—1945 гг.»'
 ),
 Museum(
     type='Монумент',
     name='«Космонавты»'
 ),
 Museum(
     type='Мемориал',
     name='«Аллея героев»'
 ),
 Museum(
     type='Мемориал',
     name='«Воинам Великой Отечественной войны — работникам ГРЭС № 5»'
 ),
 Museum(
     type='Мемориал',
     name='«Воинам-торфяникам»'
 ),
 Museum(
     type='Памятник',
     name='«Торфяница»'
 ),
 Museum(
     type='Памятник',
     name='«Зенитка»'
 ),
 Museum(
     type='Мемориал',
     name='«Воинам-интернационалистам»'
 ),
 Museum(
     type='Мемориал',
     name='«Жертвам политических репрессий»'
 ),
 Museum(
     type='Мемориал',
     name='«Ликвидаторам Чернобыля»'
 )]